In [ ]:
def minimize(X, f, length, *args):
    INT = 0.1  # don't reevaluate within 0.1 of the limit of the current bracket
    EXT = 3.0  # extrapolate maximum 3 times the current step-size
    MAX = 20  # max 20 function evaluations per line search
    RATIO = 10  # maximum allowed slope ratio
    SIG = 0.1
    RHO = SIG / 2  # SIG and RHO are the constants controlling the Wolfe-Powell conditions
    if isinstance(length, tuple) and len(length) == 2:
        red = length[1]
        length = length[0]
    else:
        red = 1

    if length > 0:
        S = 'Linesearch'
    else:
        S = 'Function evaluation'
    
    i = 0  # zero the run length counter
    ls_failed = False  # no previous line search has failed
    f0, df0 = f(X, *args)  # get function value and gradient

    Z = np.copy(X)
    X = np.unwrap(X)
    df0 = np.unwrap(df0)

    #print('Linesearch {:6}; Value {:.6e}'.format(i, f0.item()))
    fX = np.array([f0])
    i = i + (length < 0)  # count epochs?!
    s = -df0
    d0 = -np.dot(s, s)  # initial search direction (steepest) and slope
    x3 = 1.0 / (red - d0)  # initial step is 1 / (|s| + 1)

    while i < abs(length):  # while not finished
        i = i + (length > 0)  # count iterations?!
        X0 = np.copy(X)
        F0 = f0
        dF0 = df0
        if length > 0:
            M = MAX
        else:
            M = min(MAX, -length - i)

        while True:  # keep extrapolating as long as necessary
            x2 = 0.0
            f2 = f0
            d2 = d0
            f3 = f0
            df3 = df0
            success = False

            while not success and M > 0:
                try:
                    M = M - 1
                    i = i + (length < 0)  # count epochs?!
                    f3, df3 = f(rewrap(Z, X + x3 * s), *args)
                    df3 = np.unwrap(df3)
                    if np.isnan(f3) or np.isinf(f3) or np.any(np.isnan(df3) + np.isinf(df3)):
                        raise ValueError
                    success = True
                except:  # catch any error which occurred in f
                    x3 = (x2 + x3) / 2  # bisect and try again

            if f3 < F0:
                X0 = X + x3 * s
                F0 = f3
                dF0 = df3  # keep best values

            d3 = np.dot(df3, s)  # new slope

            if np.any(d3 > (SIG * d0)) or np.any(f3 > (f0 + x3 * RHO * d0)) or M == 0:
                break

            x1 = x2
            f1 = f2
            d1 = d2  # move point 2 to point 1
            x2 = x3
            f2 = f3
            d2 = d3  # move point 3 to point 2
            A = 6 * (f1 - f2) + 3 * (d2 + d1) * (x2 - x1)  # make cubic extrapolation
            B = 3 * (f2 - f1) - (2 * d1 + d2) * (x2 - x1)
            x3 = x1 - d1 * (x2 - x1) ** 2 / (B + np.sqrt(B * B - A * d1 * (x2 - x1)))  # num. error possible, ok!

            if np.isnan(x3) or np.isinf(x3) or x3 < 0:  # num prob | wrong sign?
                x3 = x2 * EXT  # extrapolate maximum amount
            elif x3 > x2 * EXT:  # new point beyond extrapolation limit?
                x3 = x2 * EXT  # extrapolate maximum amount
            elif x3 < x2 + INT * (x2 - x1):  # new point too close to previous point?
                x3 = x2 + INT * (x2 - x1)

        while (np.any(abs(d3) > -SIG * d0) or np.any(f3 > f0 + x3 * RHO * d0)) and M > 0:  # keep interpolating
            if d3 > 0 or f3 > f0 + x3 * RHO * d0:  # choose subinterval
                x4 = x3
                f4 = f3
                d4 = d3  # move point 3 to point 4
            else:
                x2 = x3
                f2 = f3
                d2 = d3  # move point 3 to point 2

            if f4 > f0:
                x3 = x2 - 0.5 * (d2 * (x4 - x2) ** 2) / (f4 - f2 - d2 * (x4 - x2))  # quadratic interpolation
            else:
                A = 6 * (f2 - f4) / (x4 - x2) + 3 * (d4 + d2)  # cubic interpolation
                B = 3 * (f4 - f2) - (2 * d2 + d4) * (x4 - x2)
                x3 = x2 + (np.sqrt(B * B - A * d2 * (x4 - x2) ** 2) - B) / A

            if np.isnan(x3) or np.isinf(x3):
                x3 = (x2 + x4) / 2  # if we had a numerical problem then bisect

            x3 = max(min(x3, x4 - INT * (x4 - x2)), x2 + INT * (x4 - x2))  # don't accept too close

            f3, df3 = f(rewrap(Z, X + x3 * s), *args)
            df3 = np.unwrap(df3)

            if f3 < F0:
                X0 = X + x3 * s
                F0 = f3
                dF0 = df3  # keep best values

            M = M - 1
            i = i + (length < 0)  # count epochs
            d3 = np.dot(df3, s)  # new slope

        if np.any(abs(d3) < -SIG * d0) and np.any(f3 < f0 + x3 * RHO * d0):  # if line search succeeded
            X = X + x3 * s  # update variables
            f0 = f3
            #print('Linesearch {:6}; Value {:.6e}'.format(i, f0.item()))
            fX = np.append(fX, f0)
            s = (np.dot(df3, df3) - np.dot(df0, df3)) / np.dot(df0, df0) * s - df3
            df0 = df3  # Polack-Ribiere CG direction

            if df0.T * s > 0 or np.isnan(f0) or np.isinf(f0):  # new slope must be negative
                s = -df0  # otherwise use steepest direction
                df0 = -np.dot(s, s)
            x3 = x3 * min(RATIO, -SIG * d0 / (d3 - SIG * d0))  # slope ratio but max RATIO
            ls_failed = False  # this line search did not fail
        else:
            X = X0
            f0 = F0
            df0 = dF0  # restore best point so far
            if ls_failed or i > abs(length):  # line search failed twice in a row
                break  # or we ran out of time, so we give up

            s = -df0
            df0 = -np.dot(s, s)  # try steepest
            x3 = 1.0 / (1 - df0)  # we increase the step size

            ls_failed = True  # line search failed
        Z = rewrap(Z, X)
    return np.concatenate(Z), fX,i



